In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def likelihood_function(wildcard, list_of_abundances, elements_to_trace, **keyword_parameters):    
    '''
    This function calculates analytically an optimal model error and the resulting likelihood from the comparison of predictions and observations

    INPUT:
        list_of_abundances = a list of the abundances coming from Chempy
        elements_to_trace = a list of the elemental symbols

    OUTPUT:
        likelihood = the added log likelihood

        element_list = the elements that were in common between the predictions and the observations, has the same sequence as the following arrays
   
        model_error = the analytic optimal model error

        star_error_list = the observed error
   
       abundance_list = the predictions
   
       star_abundance_list = the observations
    '''
    #try:
    #    wildcard = np.load(stellar_identifier + '.npy')
    #except Exception as ex:
    #    from . import localpath
    #    wildcard = np.load(localpath + 'input/stars/' + stellar_identifier + '.npy')
    # Brings the model_abundances, the stellar abundances and the associated error into the same sequence
    abundance_list = []
    element_list = []
    star_abundance_list = []
    star_error_list = []
    for i,item in enumerate(elements_to_trace):
        if item in list(wildcard.dtype.names):
            element_list.append(item)
            abundance_list.append(float(list_of_abundances[i]))
            star_abundance_list.append(float(wildcard[item][0]))
            star_error_list.append(float(wildcard[item][1]))
    abundance_list = np.hstack(abundance_list)
    star_abundance_list = np.hstack(star_abundance_list)
    star_error_list = np.hstack(star_error_list)
    assert len(abundance_list) == len(star_abundance_list) == len(star_error_list), 'no equal length, something went wrong'
    ###################

    # Introduces the optimal model error which can be derived analytically
    if ('fixed_model_error' in keyword_parameters):
        fixed_model_error = keyword_parameters['fixed_model_error']
        elements = keyword_parameters['elements']
        el_list = []
        for i,item in enumerate(elements):
            el_list.append(item)#.decode('utf-8'))
        elements = np.hstack(el_list)
        model_error = []
        for i,item in enumerate(element_list):
            model_error.append(fixed_model_error[np.where(elements == item)])
        model_error = np.hstack(model_error)
    else:
        model_error = []
        for i, item in enumerate(element_list):
            if (abundance_list[i] - star_abundance_list[i]) * (abundance_list[i] - star_abundance_list[i]) <= star_error_list[i] * star_error_list[i]:
                model_error.append(0.)
            else:
                model_error.append(np.sqrt((abundance_list[i] - star_abundance_list[i]) * (abundance_list[i] - star_abundance_list[i]) - star_error_list[i] * star_error_list[i]))
        model_error = np.hstack(model_error)

    ## Uncomment next line if you do not want to use model errors
    #model_error = np.zeros_like(model_error)
    
    # This is the best model error at the median posterior of the Proto-sun (just used to see how this effects the posterior distribution)
    #model_error = np.array([ 0.0780355, 0., 0.15495525, 0.00545988,  0.3063154,   0., 0.1057009,  0.05165564,  0., 0.72038212,  0., 0.08926388,  0., 0.27583715,  0.22945499,  0.09774014,  0.17965589 , 0. ,0.17686723,  0.21137374,  0.37973184,  0.2263486 ])
    
    # Now the likelihood is evaluated
    
    # Test to check correct abundances fed in	
    #print('Abundance List')
    #print(abundance_list)
    
    likelihood = likelihood_evaluation(model_error, star_error_list, abundance_list, star_abundance_list)
    return likelihood, element_list, model_error, star_error_list, abundance_list, star_abundance_list


In [ ]:
from Chempy.cem_function import cem_real2,extract_parameters_and_priors,posterior_function_returning_predictions
from Chempy.parameter import ModelParameters
from Chempy.data_to_test import likelihood_evaluation
changing_parameter = [-3.,-3.,-0.8,-0.3,0.5,0.6]
a = ModelParameters()
stellar_identifier = a.stellar_identifier

# the values in a are updated according to changing_parameters and the prior list is appended
a = extract_parameters_and_priors(changing_parameter, a)

# the log prior is calculated
prior = sum(np.log(a.prior))
_,elements = posterior_function_returning_predictions((changing_parameter,a))

# call Chempy and return the abundances at the end of the simulation = time of star's birth and the corresponding element names as a list
abundance_list,elements_to_trace = cem_real2(a)

# The last two entries of the abundance list are the Corona metallicity and the SN-ratio
abundance_list = abundance_list[:-2]
elements_to_trace = elements_to_trace[:-2]

# a likelihood is calculated where the model error is optimized analytically if you do not want model error uncomment one line in the likelihood function
from scipy.stats import beta
likelihood_list = []
model_errors = np.linspace(a.flat_model_error_prior[0],a.flat_model_error_prior[1],a.flat_model_error_prior[2])
error_weight = beta.pdf(model_errors, a = a.beta_error_distribution[1], b = a.beta_error_distribution[2])
error_weight/= sum(error_weight)

error_temp = error_temp = np.ones(len(elements)) * model_errors[0] 
wildcard = np.load('Chempy/input/stars/Proto-sun.npy')

In [1]:
changing_parameter = [-3.,-3.,-0.8,-0.3,0.5,0.6]
from Chempy.cem_function import posterior_function_for_integration
%timeit posterior_function_for_integration(changing_parameter)

## DEFINITELY CAN IMPROVE THIS

1 loop, best of 3: 152 ms per loop
